# Courseroad Optimization

## Installing Packages

In [1]:
# Install packages
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("Plots")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using CSV, DataFrames, JuMP, Gurobi, Plots

## Importing and Setting up Data

In [3]:
course_data = CSV.read("course_data.csv", DataFrame)

Row,no,co,cl,hh,ha,hs,he,ci,cw,re,la,u1,u2,u3,le,sa,mw,pr,on,ra,h,si,nx,rp,f,hf,vu,fa,sp
,String7,String3,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,String1,String,String,String,String7?,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Float64,Int64,Int64
1,1.00,1,00,0,0,0,0,0,0,1,0,3,2,7,U,[''],['1.001'],"[['18.01', '18.01A']]",missing,5.57,13.33,32.75,0,0,0,0,12.0,0,1
2,1.000,1,000,0,0,0,0,0,0,1,0,3,2,7,U,[''],[''],[['18.03']],missing,5.7,9.6,13.3,0,0,1,0,12.0,1,0
3,1.001,1,001,0,0,0,0,0,0,0,0,3,2,7,G,[''],['1.00'],"[['18.01', '18.01A']]",missing,5.57,13.33,32.75,0,0,0,0,12.0,0,1
4,1.005,1,005,0,0,0,0,0,0,0,0,1,0,2,U,[''],[''],[],missing,6.1,2.7,19.0,0,0,0,0,3.0,1,0
5,1.008,1,008,0,0,0,0,0,0,0,0,1,1,1,U,[''],[''],[],missing,6.5,2.4,13.0,0,0,0,0,3.0,1,0
6,1.009,1,009,0,0,0,0,0,0,0,0,1,0,2,U,[''],[''],[],missing,5.7,2.8,27.0,0,0,0,0,3.0,1,0
7,1.010,1,010,0,0,0,0,0,0,0,0,5,0,7,U,[''],[''],"[['18.02', '18.02A', '18.022']]",missing,5.4,8.8,21.0,0,0,0,0,12.0,1,0
8,1.013,1,013,0,0,0,0,0,0,0,0,1,3,2,U,[''],[''],[],missing,3.52,11.17,19.5,0,1,0,0,6.0,1,1
9,1.015,1,015,0,0,0,0,0,0,0,0,3,3,6,U,['2.017'],[''],"[['2.003'], ['2.016'], ['2.678']]",missing,5.8,13.26,18.75,0,0,0,0,12.0,0,1


In [14]:
# Build sets
classes = 1:nrow(course_data)
semesters = 1:8

3535-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

### Data Directly from CSV

In [64]:
# Get basic class information
names = course_data."no"
fall = course_data."fa"
spring = course_data."sp"
U = course_data."vu" # units
R = course_data."ra" # ratings
H = course_data."h" # hours


# Get GIR-related information
hh = course_data."hh"
ha = course_data."ha"
hs = course_data."hs"
he = course_data."he"
cih = course_data."ci"
ci_hw = course_data."cw"
rest = course_data."re"
lab = course_data."la"

3535-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

### Data Processing

In [60]:
# Determining which classes are hasses
hass = []
for i in classes
    if hass_h[i] == 1 | hass_a[i] == 1 | hass_s[i] == 1 | hass_e[i] == 1
        push!(hass, 1)
    else
        push!(hass, 0)
    end
end

# Enumerating classes satisfying each specific GIR requirement
satisfying_bio = ["7.012", "7.013", "7.014", "7.015", "7.016"]

satisfying_chem = ["3.091", "5.111", "5.112"]

satisfying_phys1 = ["8.01", "8.011", "8.012", "8.01L"]
satisfying_phys2 = ["8.02", "8.021", "8.022"]

satisfying_calc1 = ["18.01", "18.01A"]
satisfying_calc2 = ["18.01", "18.01A"]

# Building one-hot vectors
bio = [names[i] in satisfying_bio ? 1 : 0 for i in classes]
chem = [names[i] in satisfying_chem ? 1 : 0 for i in classes]
phys1 = [names[i] in satisfying_phys1 ? 1 : 0 for i in classes]
phys2 = [names[i] in satisfying_phys2 ? 1 : 0 for i in classes]
calc1 = [names[i] in satisfying_calc1 ? 1 : 0 for i in classes]
calc2 = [names[i] in satisfying_calc2 ? 1 : 0 for i in classes]


# Making matrix of GIR requirements 
G = hcat(ha, hs, hh, cih, hass, bio, phys1, phys2, calc1, calc2, chem, rest, lab)
G_sat = [1, 1, 1, 2, 8, 1, 1, 1, 1, 1, 1, 12, 12]


# Making matrix for fall/spring classes
A = hcat(fall, spring)

# Make vector indicating whether a semester is a fall (1) or spring (2) semester
S = [ (j%2 == 1) ? 1 : 2 for j in semesters]

8-element Vector{Int64}:
 1
 2
 1
 2
 1
 2
 1
 2

## Tweakable variables

In [63]:
alpha = 0.5

0.5

## Building the Model

### Index references
i: class

j: semester

k: index of GIR requirement

### Model Definition

In [65]:
model = Model(Gurobi.Optimizer)

# Decision variables
@variable(model, X[classes, semesters], Bin);

# Objective
@objective(model, Min, alpha*sum(H[i]*sum(X[i,j] for j in semesters) for i in classes) - (1- alpha)*sum(R[i]*sum(X[i,j] for j in semesters) for i in classes));

# ---- General Constraints ------

# Cannot take a class more than once
@constraint(model, num_times[i in classes], sum(X[i,j] for j in semesters) <= 1);

# Maximum and minimum number of units per semester
@constraint(model, max_units[j in semesters], sum(U[i]*X[i,j] for i in classes) <= 60);
@constraint(model, min_units[j in semesters], sum(U[i]*X[i,j] for i in classes) >= 36);

# Fall classes taken in fall, spring classes taken in spring
@constraint(model, fall_spring[i in classes, j in semesters], X[i,j] - A[i, S[j]] <= 0);

# ---- GIR Constraints ------



Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
